In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import datetime
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.austintexas.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fdj4-gpfu", limit=200000,)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [2]:
results_df

,incident_report_number,crime_type,ucr_code,family_violence,occ_date_time,occ_date,occ_time,rep_date_time,rep_date,rep_time,...,longitude,location,:@computed_region_a3it_2a2z,:@computed_region_8spj_utxs,:@computed_region_q9nd_rr82,:@computed_region_qwte_z96m,ucr_category,category_description,clearance_status,clearance_date
0,202211675,DWI,2100,N,2022-01-01T23:52:00.000,2022-01-01T00:00:00.000,2352,2022-01-02T01:15:00.000,2022-01-02T00:00:00.000,115,...,-97.71818055,"{'latitude': '30.28445156', 'longitude': '-97....",2865,9,10,1169,NaN,NaN,NaN,NaN
1,202211497,CRUELTY TO ANIMALS,2717,N,2022-01-01T23:50:00.000,2022-01-01T00:00:00.000,2350,2022-01-01T23:50:00.000,2022-01-01T00:00:00.000,2350,...,-97.78766592,"{'latitude': '30.25858957', 'longitude': '-97....",3260,8,2,NaN,NaN,NaN,NaN,NaN
2,202211500,FAMILY DISTURBANCE,3400,N,2022-01-01T23:47:00.000,2022-01-01T00:00:00.000,2347,2022-01-01T23:47:00.000,2022-01-01T00:00:00.000,2347,...,-97.72000474,"{'latitude': '30.3698269', 'longitude': '-97.7...",3646,7,7,NaN,NaN,NaN,NaN,NaN
3,202211364,AGG ASLT STRANGLE/SUFFOCATE,410,Y,2022-01-01T23:24:00.000,2022-01-01T00:00:00.000,2324,2022-01-01T23:24:00.000,2022-01-01T00:00:00.000,2324,...,-97.77442753,"{'latitude': '30.22637821', 'longitude': '-97....",3259,3,3,200,13A,Aggravated Assault,NaN,NaN
4,2022220011633,ASSAULT W/INJURY-FAM/DATE VIOL,900,N,2022-01-01T23:18:00.000,2022-01-01T00:00:00.000,2318,2022-01-01T23:18:00.000,2022-01-01T00:00:00.000,2318,...,-97.71651521,"{'latitude': '30.23973812', 'longitude': '-97....",3256,3,3,102,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,20193441340,HARASSMENT OF A PUBLIC SERVANT,2731,N,2019-12-10T17:39:00.000,2019-12-10T00:00:00.000,1739,2019-12-10T17:48:00.000,2019-12-10T00:00:00.000,1748,...,-97.73761583,"{'latitude': '30.2679761', 'longitude': '-97.7...",2856,9,10,NaN,NaN,NaN,C,2019-12-10T00:00:00.000
199996,20193441366,POSSESSION OF MARIJUANA,1803,N,2019-12-10T17:38:00.000,2019-12-10T00:00:00.000,1738,2019-12-10T17:38:00.000,2019-12-10T00:00:00.000,1738,...,-97.67918239,"{'latitude': '30.39725993', 'longitude': '-97....",3642,7,7,NaN,NaN,NaN,C,2019-12-10T00:00:00.000
199997,20193441356,FALSE CALLER ID DISPLAY/SPOOF,2736,N,2019-12-10T17:31:00.000,2019-12-10T00:00:00.000,1731,2019-12-10T17:31:00.000,2019-12-10T00:00:00.000,1731,...,-97.77878034,"{'latitude': '30.16358776', 'longitude': '-97....",3258,5,1,3631,NaN,NaN,N,2020-01-13T00:00:00.000
199998,20198005092,THEFT BY SHOPLIFTING,607,N,2019-12-10T17:15:00.000,2019-12-10T00:00:00.000,1715,2019-12-18T11:40:00.000,2019-12-18T00:00:00.000,1140,...,-97.74836248,"{'latitude': '30.3985376', 'longitude': '-97.7...",3647,10,5,NaN,23C,Theft,N,2019-12-31T00:00:00.000


In [3]:
results_df.dtypes

incident_report_number         object
crime_type                     object
ucr_code                       object
family_violence                object
occ_date_time                  object
occ_date                       object
occ_time                       object
rep_date_time                  object
rep_date                       object
rep_time                       object
location_type                  object
address                        object
zip_code                       object
council_district               object
sector                         object
district                       object
pra                            object
census_tract                   object
x_coordinate                   object
y_coordinate                   object
latitude                       object
longitude                      object
location                       object
:@computed_region_a3it_2a2z    object
:@computed_region_8spj_utxs    object
:@computed_region_q9nd_rr82    object
:@computed_r

In [25]:
crime_df = results_df[['category_description', 'family_violence', 'crime_type', 'occ_date', 'occ_time', 'location_type','address', 'zip_code', 'latitude', 'longitude']]
crime_df.dropna(inplace=True)
crime_df.sample(10)

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,category_description,family_violence,crime_type,occ_date,occ_time,location_type,address,zip_code,latitude,longitude
154580,Theft,N,THEFT,2020-05-20T00:00:00.000,1200,DEPARTMENT / DISCOUNT STORE,12901 N IH 35 SVRD NB,78753,30.41525799,-97.67013132
19103,Theft,N,THEFT,2021-10-10T00:00:00.000,200,BAR / NIGHT CLUB,710 W 6TH ST,78701,30.27025454,-97.75005784
186357,Robbery,N,ROBBERY BY THREAT,2020-01-28T00:00:00.000,142,CONVENIENCE STORE,13201 N FM 620 RD NB,78729,30.46793555,-97.79250733
120818,Theft,N,BURGLARY OF VEHICLE,2020-09-19T00:00:00.000,2000,RESIDENCE / HOME,7101 VILLA MARIA LN,78759,30.40808775,-97.77710228
197871,Theft,N,BURGLARY OF VEHICLE,2019-12-18T00:00:00.000,1200,COMMERCIAL / OFFICE BUILDING,6121 N LAMAR BLVD,78752,30.3299944,-97.72369939
47264,Burglary,N,BURGLARY NON RESIDENCE,2021-06-20T00:00:00.000,1730,GROCERY / SUPERMARKET,7010 EASY WIND DR,78752,30.33918678,-97.71833294
153294,Auto Theft,N,AUTO THEFT,2020-05-24T00:00:00.000,1800,PARKING LOTS / GARAGE,2430 CROMWELL CIR,78741,30.22664294,-97.72105703
57589,Auto Theft,N,AUTO THEFT,2021-05-10T00:00:00.000,1258,AUTO DEALERSHIP NEW / USED,409 E BEN WHITE BLVD EB,78704,30.22053579,-97.75984017
96320,Theft,N,THEFT,2020-12-15T00:00:00.000,1600,RESIDENCE / HOME,1500 E PARMER LN,78753,30.38979111,-97.65120601
73403,Theft,N,BURGLARY OF VEH-NO SUSPECT/FU,2021-03-13T00:00:00.000,1405,STREETS / HWY / ROAD / ALLEY,5604 SOUTHWEST PKWY,78735,30.24790772,-97.84521842


In [26]:
#Converting time data to datetime

timefmt_date = "%Y-%m-%d"
timefmt_time = "%H:%M" 
crime_df['occ_date']= pd.to_datetime(crime_df['occ_date'], format = timefmt_date, errors='coerce')

time_occ = []

# Loop to format data for date time
for str in crime_df['occ_time']:
          if len(str) == 4 :
                    val =  str[:2] + ':' +str[2:]
          elif len(str) == 3:
                    val = str[:1] + ':' +str[1:]
          elif len(str) == 1:
                    val = '00:00'
          time_occ.append(val)
#
crime_df['time_occ'] = time_occ

crime_df['time_occ'] = pd.to_datetime(crime_df['time_occ'], format = timefmt_time, errors='coerce')
crime_df['time_occ'] = [time.time() for time in crime_df['time_occ']]
          

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [27]:
crime_df.dtypes

category_description            object
family_violence                 object
crime_type                      object
occ_date                datetime64[ns]
occ_time                        object
location_type                   object
address                         object
zip_code                        object
latitude                        object
longitude                       object
time_occ                        object
dtype: object

In [28]:
crime_df = crime_df.loc[(crime_df['occ_date']>= '2020-01-01') & (crime_df['occ_date']< '2022-01-01')]
crime_df

,category_description,family_violence,crime_type,occ_date,occ_time,location_type,address,zip_code,latitude,longitude,time_occ
134,Aggravated Assault,N,DEADLY CONDUCT,2021-12-31,2320,RESIDENCE / HOME,2513 FRANCISCO ST,78702,30.26340897,-97.71187196,23:20:00
135,Theft,N,THEFT,2021-12-31,2300,RESIDENCE / HOME,3220 AMY DONOVAN PLZ,78758,30.3967807,-97.72731711,23:00:00
142,Theft,N,BURGLARY OF VEHICLE,2021-12-31,2228,PARKING LOTS / GARAGE,901 LITTLE TEXAS LN,78745,30.19818661,-97.76915525,22:28:00
145,Aggravated Assault,N,DEADLY CONDUCT,2021-12-31,2200,STREETS / HWY / ROAD / ALLEY,300 ALLEN ST,78702,30.25307883,-97.70651011,22:00:00
146,Aggravated Assault,Y,AGG ASLT STRANGLE/SUFFOCATE,2021-12-31,2149,RESIDENCE / HOME,11700 DIONDA LN,78617,30.2047188,-97.63309584,21:49:00
...,...,...,...,...,...,...,...,...,...,...,...
194054,Theft,N,THEFT,2020-01-01,0,RESIDENCE / HOME,1307 W RUNDBERG LN,78758,30.36756821,-97.70731527,00:00:00
194059,Theft,N,THEFT OF BICYCLE,2020-01-01,0,PARKING LOTS / GARAGE,507 PRESSLER ST,78703,30.27315833,-97.76014527,00:00:00
194062,Burglary,N,BURGLARY NON RESIDENCE,2020-01-01,0,RENTAL STORAGE FACILITY,405 E ST ELMO RD,78745,30.21523129,-97.76330661,00:00:00
194063,Theft,N,MAIL THEFT,2020-01-01,0,RESIDENCE / HOME,4509 E ST ELMO RD,78744,30.20549514,-97.73862021,00:00:00


In [29]:
#drop old time column. 
crime_df.drop('occ_time',axis = 1,inplace=True)

In [30]:
crime_df

,category_description,family_violence,crime_type,occ_date,location_type,address,zip_code,latitude,longitude,time_occ
134,Aggravated Assault,N,DEADLY CONDUCT,2021-12-31,RESIDENCE / HOME,2513 FRANCISCO ST,78702,30.26340897,-97.71187196,23:20:00
135,Theft,N,THEFT,2021-12-31,RESIDENCE / HOME,3220 AMY DONOVAN PLZ,78758,30.3967807,-97.72731711,23:00:00
142,Theft,N,BURGLARY OF VEHICLE,2021-12-31,PARKING LOTS / GARAGE,901 LITTLE TEXAS LN,78745,30.19818661,-97.76915525,22:28:00
145,Aggravated Assault,N,DEADLY CONDUCT,2021-12-31,STREETS / HWY / ROAD / ALLEY,300 ALLEN ST,78702,30.25307883,-97.70651011,22:00:00
146,Aggravated Assault,Y,AGG ASLT STRANGLE/SUFFOCATE,2021-12-31,RESIDENCE / HOME,11700 DIONDA LN,78617,30.2047188,-97.63309584,21:49:00
...,...,...,...,...,...,...,...,...,...,...
194054,Theft,N,THEFT,2020-01-01,RESIDENCE / HOME,1307 W RUNDBERG LN,78758,30.36756821,-97.70731527,00:00:00
194059,Theft,N,THEFT OF BICYCLE,2020-01-01,PARKING LOTS / GARAGE,507 PRESSLER ST,78703,30.27315833,-97.76014527,00:00:00
194062,Burglary,N,BURGLARY NON RESIDENCE,2020-01-01,RENTAL STORAGE FACILITY,405 E ST ELMO RD,78745,30.21523129,-97.76330661,00:00:00
194063,Theft,N,MAIL THEFT,2020-01-01,RESIDENCE / HOME,4509 E ST ELMO RD,78744,30.20549514,-97.73862021,00:00:00


In [31]:
# Convert latitude and longitude as float
crime_df['latitude'] = crime_df['latitude'].astype('float')
crime_df['longitude'] = crime_df['longitude'].astype('float')
crime_df.dtypes


category_description            object
family_violence                 object
crime_type                      object
occ_date                datetime64[ns]
location_type                   object
address                         object
zip_code                        object
latitude                       float64
longitude                      float64
time_occ                        object
dtype: object

In [32]:
# Converting zip_codes to integers
crime_df['zip_code'] = crime_df['zip_code'].astype('int64')
crime_df.dtypes


category_description            object
family_violence                 object
crime_type                      object
occ_date                datetime64[ns]
location_type                   object
address                         object
zip_code                         int64
latitude                       float64
longitude                      float64
time_occ                        object
dtype: object

In [33]:
# Reorganize column info

column_order = ['category_description', 'crime_type', 'family_violence', 'occ_date', 'time_occ', 'address', 'zip_code','latitude','longitude']
crime_df = crime_df[column_order]
crime_df.groupby(['zip_code']).count()

,category_description,crime_type,family_violence,occ_date,time_occ,address,latitude,longitude
zip_code,,,,,,,,
78610,1,1,1,1,1,1,1,1
78613,438,438,438,438,438,438,438,438
78617,408,408,408,408,408,408,408,408
78652,18,18,18,18,18,18,18,18
78653,69,69,69,69,69,69,69,69
78660,643,643,643,643,643,643,643,643
78664,1,1,1,1,1,1,1,1
78681,1,1,1,1,1,1,1,1
78701,3534,3534,3534,3534,3534,3534,3534,3534


In [34]:
#grouping crime type by zipcode

crime_by_zipcode_df = crime_df.groupby(['zip_code', 'category_description']).count()[['crime_type']]
crime_by_zipcode_df

crime_type
zip_code category_description            
78610    Theft                          1
78613    Aggravated Assault            11
         Auto Theft                    15
         Burglary                      55
         Robbery                        4
...                                   ...
78759    Auto Theft                   271
         Burglary                     370
         Murder                         3
         Robbery                       44
         Theft                       2598

[252 rows x 1 columns]

In [35]:
zip_code_pricing = pd.read_csv('Book_4.csv')
zip_code_pricing

,ZIP,DOM:\nQ1 2020,DOM:\nQ1 2021,Sale price:\nQ1 2020,Sale price:\nQ1 2021,$/sq ft:\nQ1 2020,$/sq ft:\nQ1 2021
0,76537,48,7,"$195,790","$220,000",$127,$141
1,76542,25,4,"$169,900","$187,000",$85,$101
2,76574,25,8,"$216,728","$244,950",$128,$149
3,78130,42,22,"$233,950","$280,160",$136,$148
4,78602,49,15,"$264,000","$290,518",$140,$158
...,...,...,...,...,...,...,...
60,78753,15,5,"$300,000","$385,000",$195,$230
61,78754,14,4,"$270,000","$365,000",$139,$179
62,78757,9,4,"$525,000","$650,000",$346,$392
63,78758,5,4,"$345,000","$460,500",$224,$287


In [ ]:
crime_df.to_csv('crime_data.csv', index=False)
crime_by_zipcode_df.to_csv("crime_zipcode_data.csv")